<a href="https://colab.research.google.com/github/myrah/AAI2025/blob/dev/aai/aiAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.generativeai as genai
import os
from google.colab import userdata

# --- Configuration ---
# IMPORTANT: Replace "YOUR_API_KEY" with your actual Gemini API key.
# You can also set this as an environment variable named 'GEMINI_API_KEY'

API_KEY = userdata.get('GOOGLE_API_KEY')

# --- Agent Definitions ---

class GeminiAgent:
    """A base class for a Gemini-powered agent."""
    def __init__(self, model_name="gemini-pro", system_prompt=None):
        """
        Initializes the agent.

        Args:
            model_name (str): The name of the Gemini model to use.
            system_prompt (str): A system-level instruction to guide the model's behavior.
        """
        if API_KEY == 'GOOGLE_API_KEY':
            raise ValueError("Please replace 'YOUR_API_KEY' with your actual Gemini API key.")

        genai.configure(api_key=API_KEY)

        generation_config = {
            "temperature": 0.7,
            "top_p": 1,
            "top_k": 1,
            "max_output_tokens": 2048,
        }

        self.model = genai.GenerativeModel(
            model_name=model_name,
            generation_config=generation_config,
            system_instruction=system_prompt
        )
        self.conversation = self.model.start_chat(history=[])

    def send_message(self, message):
        """Sends a message to the Gemini model and returns the response."""
        response = self.conversation.send_message(message)
        return response.text

# --- Specialist Agent Prompts ---

TRIAGE_PROMPT = """
You are a Triage Agent. Your job is to analyze the user's request and route them to the correct department.
You must respond with ONLY ONE of the following three options:
- 'Technical Support'
- 'Billing'
- 'General Inquiry'
"""

TECHNICAL_SUPPORT_PROMPT = """
You are a friendly and helpful Technical Support Agent.
Your goal is to patiently and clearly guide the user to solve their technical issue.
Start by acknowledging their problem and then ask for specific details.
"""

BILLING_PROMPT = """
You are a professional and courteous Billing Agent.
Your goal is to help users with their billing, payment, or subscription questions.
Start by politely asking for an account number or invoice ID to locate their information.
"""


# --- Main Application Logic ---

def main():
    """The main function to run the multi-agent chatbot."""

    print("🤖 Chatbot initialized. Welcome to our support center!")
    print("🤖 You can type 'quit' at any time to exit.")

    # 1. GREETING
    # The initial greeting is handled by the main script.
    initial_prompt = "How can I help you today?"
    print(f"\nGreeter Agent: {initial_prompt}")

    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Goodbye!")
        return

    # 2. TRIAGE
    # The Triage Agent decides where to send the user.
    print("\nSystem: Analyzing your request and finding the right agent...")
    triage_agent = GeminiAgent(system_prompt=TRIAGE_PROMPT)

    # We only send the user's first message to the Triage agent.
    try:
        route = triage_agent.send_message(user_input).strip().lower()
    except Exception as e:
        print(f"\nSystem: Sorry, there was an error with our routing agent: {e}")
        return

    specialist_agent = None
    agent_name = ""

    # 3. ROUTING
    # Based on the triage result, select the correct specialist agent.
    if 'technical support' in route:
        print("System: Routing to Technical Support...")
        agent_name = "Technical Support Agent"
        specialist_agent = GeminiAgent(system_prompt=TECHNICAL_SUPPORT_PROMPT)
    elif 'billing' in route:
        print("System: Routing to Billing Department...")
        agent_name = "Billing Agent"
        specialist_agent = GeminiAgent(system_prompt=BILLING_PROMPT)
    else:
        print("System: Routing to General Inquiry...")
        agent_name = "General Inquiry Agent"
        # Using a default agent with no special prompt
        specialist_agent = GeminiAgent(system_prompt="You are a helpful general inquiry agent.")

    # 4. CONVERSATION with the Specialist
    # The specialist agent takes over the conversation.

    # The specialist needs the user's first message to have context.
    try:
        response = specialist_agent.send_message(user_input)
        print(f"\n{agent_name}: {response}")
    except Exception as e:
        print(f"\nSystem: Sorry, there was an error with our specialist agent: {e}")
        return


    # Loop for continuous conversation with the specialist
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Thank you for chatting with us. Goodbye!")
            break

        try:
            response = specialist_agent.send_message(user_input)
            print(f"\n{agent_name}: {response}")
        except Exception as e:
            print(f"\nSystem: An error occurred: {e}")
            break


if __name__ == "__main__":
    main()

🤖 Chatbot initialized. Welcome to our support center!
🤖 You can type 'quit' at any time to exit.

Greeter Agent: How can I help you today?
You: billing

System: Analyzing your request and finding the right agent...



System: Sorry, there was an error with our routing agent: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
